In [76]:
# 再度コードを実行するための準備
import pandas as pd
import numpy as np

# ファイルのパス
file_path = '中間成果物/所在管理MBデータ_統合済&特定日時抽出済&手配数と手配運用情報統合済.csv'

# CSVファイルの読み込み（エンコーディングを cp932 に設定）
try:
    data = pd.read_csv(file_path, encoding='cp932')
except Exception as e:
    print(e)

# '検収日時'カラムを日時型に変換
data['検収日時'] = pd.to_datetime(data['検収日時'], errors='coerce')

# 日時型に変換したカラムから日付の部分を抽出
data['検収日'] = data['検収日時'].dt.day

#①品番と納入便ごとにかんばん数を計算
kanban_count = data.groupby(['仕入先名','品番','検収日','納入便','納入回数（間隔）','納入回数（回数）','納入回数（遅れ）']).size().reset_index(name='かんばん数')
bin_count = data.groupby(['仕入先名','品番','納入回数（間隔）','納入回数（回数）','納入回数（遅れ）'])['日量数（箱数）'].max().reset_index(name='最大日量数（箱数）')
nichiryo_count = data.groupby(['仕入先名','品番', '検収日','納入回数（間隔）','納入回数（回数）','納入回数（遅れ）'])['日量数（箱数）'].mean().reset_index(name='日量数（箱数）2')
# 品番と検収日ごとに納入便のユニークな値の数を計算
unique_deliveries = data.groupby(['品番', '検収日'])['納入便'].nunique().reset_index(name='ユニークな納入便の数')

result = pd.merge(kanban_count, bin_count, on=['仕入先名','品番','納入回数（間隔）','納入回数（回数）','納入回数（遅れ）'], how='outer')
result = pd.merge(result, nichiryo_count, on=['仕入先名','品番', '検収日','納入回数（間隔）','納入回数（回数）','納入回数（遅れ）'], how='outer')
result = pd.merge(result, unique_deliveries, on=['品番', '検収日'], how='outer')

#------ピッチと結合
pitch = pd.read_csv('中間成果物/不等ピッチデータ_統合済.csv', encoding='Shift_JIS')
longi = len(result.loc[:,"品番"])
longj = len(pitch.loc[:,"仕入先名"])

for i in range(longi):
    for j in range(longj):
        if (result.loc[i, '仕入先名'] == pitch.loc[j,'仕入先名']):
            result.loc[i,"不等ピッチ係数"] = pitch.loc[j,"不等ピッチ係数【日】"]
            break
            
#-----終了

print("end")

#------在庫と結合
zaiko = pd.read_csv('中間成果物/在庫推移MBデータ_統合済.csv', encoding='Shift_JIS')
# '検収日時'カラムを日時型に変換
zaiko['計測日時'] = pd.to_datetime(zaiko['計測日時'], errors='coerce')
# 日時型に変換したカラムから日付の部分を抽出
zaiko['検収日'] = zaiko['計測日時'].dt.day

zaiko2 = zaiko.groupby(['品番','検収日'])['在庫数（箱）'].mean().reset_index(name='日在庫数（箱）')
zaiko3 = zaiko.groupby(['品番','検収日'])['入庫数（箱）'].sum().reset_index(name='合計入庫数（箱）')
zaiko4 = zaiko.groupby(['品番','検収日'])['出庫数（箱）'].sum().reset_index(name='合計出庫数（箱）')

result = pd.merge(result, zaiko2, on=['品番','検収日'], how='outer')
result = pd.merge(result, zaiko3, on=['品番','検収日'], how='outer')
result = pd.merge(result, zaiko4, on=['品番','検収日'], how='outer')
            
#-----終了

result['不等ピッチ在庫']=result['不等ピッチ係数']*result['最大日量数（箱数）']
result['最大便Ave']=result['最大日量数（箱数）']/result['納入回数（回数）']
result['便Ave+不等ピッチ在庫']=result['最大便Ave']+result['不等ピッチ在庫']
result['差分']=result['かんばん数']-result['便Ave+不等ピッチ在庫']

result['設計値MIN'] = np.ceil(0.1*(result['最大日量数（箱数）']*result['納入回数（間隔）']*(1+result['納入回数（遅れ）'])/result['納入回数（回数）']))
result['設計値MAX'] = result['設計値MIN'] + np.ceil(result['便Ave+不等ピッチ在庫'])
result['在庫数/設計値MAX'] = result['日在庫数（箱）']/result['設計値MAX']

with open("kari2.csv", mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    result.to_csv(f)
    
# 計算結果の確認
display(result)

end


,仕入先名,品番,検収日,納入便,納入回数（間隔）,納入回数（回数）,納入回数（遅れ）,かんばん数,最大日量数（箱数）,日量数（箱数）2,...,日在庫数（箱）,合計入庫数（箱）,合計出庫数（箱）,不等ピッチ在庫,最大便Ave,便Ave+不等ピッチ在庫,差分,設計値MIN,設計値MAX,在庫数/設計値MAX
0,アイコー（株）,3559850A010,1,1.0,1.0,2.0,3.92,1.0,3.77,3.37,...,4.666667,3.0,4.0,1.2064,1.885,3.0914,-2.0914,1.0,5.0,0.933333
1,アイコー（株）,3559850A010,1,2.0,1.0,2.0,3.92,2.0,3.77,3.37,...,4.666667,3.0,4.0,1.2064,1.885,3.0914,-1.0914,1.0,5.0,0.933333
2,アイコー（株）,3559850A010,4,1.0,1.0,2.0,3.92,1.0,3.77,3.30,...,3.541667,1.0,2.0,1.2064,1.885,3.0914,-2.0914,1.0,5.0,0.708333
3,アイコー（株）,3559850A010,5,2.0,1.0,2.0,3.92,2.0,3.77,3.26,...,3.250000,3.0,3.0,1.2064,1.885,3.0914,-1.0914,1.0,5.0,0.650000
4,アイコー（株）,3559850A010,6,1.0,1.0,2.0,3.92,2.0,3.77,3.24,...,3.541667,5.0,4.0,1.2064,1.885,3.0914,-1.0914,1.0,5.0,0.708333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11490,NaN,Y021782011*****,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11491,NaN,Y021782011*****,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11492,NaN,Y021782011*****,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.250000,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11493,NaN,Y021782011*****,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.529412,37.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
